## Filter elo

In [11]:
# Filtering
min_elo = 2000

import pandas as pd

games = pd.read_csv('./data/games_filtered_1.csv')
print(games)

games = games[
    (games['winner'] == 'white') & (games['white_rating'] > min_elo) |
    (games['winner'] == 'black') & (games['black_rating'] > min_elo)
    ]
games = games.reset_index(drop=True)
games = games[['winner', 'white_rating', 'black_rating', 'moves']]
print(games)

games.to_csv("./data/games_"+str(min_elo)+".csv")

      winner  white_rating  black_rating  \
0      white          1500          1191   
1      black          1322          1261   
2      white          1496          1500   
3      white          1439          1454   
4      white          1523          1469   
...      ...           ...           ...   
20053  white          1691          1220   
20054  black          1233          1196   
20055  white          1219          1286   
20056  white          1360          1227   
20057  black          1235          1339   

                                                   moves  
0      d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...  
1      d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...  
2      e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...  
3      d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...  
4      e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...  
...                                                  ...  
20053  d4 f5 e3 e6 Nf3 Nf6 Nc3 b6 Be2 Bb7 O-O Be7 Ne5...  

## Load Filtered data

In [6]:
import pandas as pd

games = pd.read_csv('./data/games_'+str(min_elo)+'.csv')
print(games)

      Unnamed: 0 winner  white_rating  black_rating  \
0              0  black          1413          2108   
1              1  white          2449           978   
2              2  white          2443           978   
3              3  white          2436           978   
4              4  white          2436           978   
...          ...    ...           ...           ...   
2180        2180  black          2094          2228   
2181        2181  black          2162          2214   
2182        2182  black          2236          2148   
2183        2183  white          2268          2254   
2184        2184  white          2049          1882   

                                                  moves  
0                        e4 Nc6 d4 e5 d5 Nce7 c3 Ng6 b4  
1     c4 e5 Nc3 Bc5 g3 Nc6 Bg2 Nf6 Nf3 b6 Nxe5 Bb7 N...  
2     e4 c5 Nf3 d6 d4 cxd4 Nxd4 Nf6 Nc3 g6 Be3 Bg7 f...  
3     e4 c5 Nf3 d6 d4 cxd4 Nxd4 Nf6 Nc3 g6 Be3 Bg7 f...  
4     e4 d5 exd5 Qxd5 Qf3 Qe5+ Ne2 Nf6 d4 Qd5 Qxd

## Set up dataframe

In [7]:
# columns titles
positions = ['p'+str(i) for i in range(64)]
columns = positions
columns += ['player']
columns += ['move']
print(columns)

['p0', 'p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12', 'p13', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p20', 'p21', 'p22', 'p23', 'p24', 'p25', 'p26', 'p27', 'p28', 'p29', 'p30', 'p31', 'p32', 'p33', 'p34', 'p35', 'p36', 'p37', 'p38', 'p39', 'p40', 'p41', 'p42', 'p43', 'p44', 'p45', 'p46', 'p47', 'p48', 'p49', 'p50', 'p51', 'p52', 'p53', 'p54', 'p55', 'p56', 'p57', 'p58', 'p59', 'p60', 'p61', 'p62', 'p63', 'player', 'move']


## Final filtering

In [8]:
from chess_transformation import *

games_transformed = pd.DataFrame(columns=columns)

index = 0
for i in range(len(games)):
# for i in range(1):
    print(str(i)+'/'+str(len(games)), end='\r')
    boards = []
    move_list = []
    game = games.iloc[i]

    moves = game['moves'].split(' ')
    winner = game['winner']
    # print(game['moves'])
    # print(moves)
    # print(winner)

    for i in range(len(moves)-1):
        if winner == 'white' and (i%2==0 or i == 0):
            boards.append(algebraic_to_matrix_python_chess(moves[:i]))
            move_list.append(moves[i])
        elif winner == 'black' and i%2==1 and i != 0:
            boards.append(algebraic_to_matrix_python_chess(moves[:i]))
            move_list.append(moves[i])

    for i in range(len(boards)):
        # print(board_to_linear(boards[i])+[winner]+[move_list[i]])
        # print('----------------------------------------')
        # print_board(boards[i])
        # print('next:', move_list[i])
        games_transformed.loc[index] = board_to_linear(boards[i])+[winner]+[move_list[i]]
        index+=1

        if index%100 == 0: # save every 100 games
            games_transformed.to_csv('./data/games_cleaned_'+str(min_elo)+'.csv')

# games_transformed.to_csv("test.csv")
games_transformed.to_csv('./data/games_cleaned_'+str(min_elo)+'.csv')
print(games_transformed)

      p0 p1 p2 p3 p4 p5 p6 p7 p8 p9  ... p56 p57 p58 p59 p60 p61 p62 p63  \
0      R  N  B  Q  K  B  N  R  P  P  ...   r   n   b   q   k   b   n   r   
1      R  N  B  Q  K  B  N  R  P  P  ...   r   .   b   q   k   b   n   r   
2      R  N  B  Q  K  B  N  R  P  P  ...   r   .   b   q   k   b   n   r   
3      R  N  B  Q  K  B  N  R  P  P  ...   r   .   b   q   k   b   n   r   
4      R  N  B  Q  K  B  N  R  P  P  ...   r   n   b   q   k   b   n   r   
...   .. .. .. .. .. .. .. .. .. ..  ...  ..  ..  ..  ..  ..  ..  ..  ..   
66594  R  .  B  Q  R  .  K  .  P  P  ...   r   .   .   .   .   .   .   r   
66595  R  .  B  Q  R  .  K  .  P  P  ...   r   .   .   .   .   .   .   r   
66596  R  .  B  Q  R  .  K  .  P  P  ...   r   .   .   .   .   .   .   r   
66597  R  .  B  Q  R  .  K  .  P  P  ...   r   .   .   .   .   .   r   .   
66598  R  .  B  .  R  .  K  .  P  P  ...   r   .   .   .   .   .   r   .   

      player   move  
0      black    Nc6  
1      black     e5  
2      black   Nce7  

## Scrambling

In [13]:
games_cleaned = pd.read_csv('./data/games_cleaned_'+str(min_elo)+'.csv')
games_cleaned = games_cleaned.sample(frac = 1).reset_index(drop=True)
print(games_cleaned)
games_cleaned.to_csv('./data/games_cleaned_'+str(min_elo)+'_scambled.csv')


       Unnamed: 0 p0 p1 p2 p3 p4 p5 p6 p7 p8  ... p56 p57 p58 p59 p60 p61 p62  \
0           34772  .  .  K  .  .  r  R  R  P  ...   .   .   .   .   .   r   .   
1            2658  .  .  .  .  .  R  K  .  .  ...   r   .   .   .   .   .   k   
2           48434  R  .  B  .  .  R  .  K  P  ...   .   .   .   .   .   r   .   
3           30771  .  .  B  .  .  R  K  .  q  ...   .   .   r   .   .   .   k   
4           12955  R  .  .  Q  K  .  .  R  P  ...   .   .   k   r   .   b   .   
...           ... .. .. .. .. .. .. .. .. ..  ...  ..  ..  ..  ..  ..  ..  ..   
66594       16113  R  .  .  Q  .  R  K  .  .  ...   .   r   .   .   .   .   k   
66595       50259  R  .  B  Q  K  B  .  R  P  ...   r   .   b   q   k   b   .   
66596       36974  R  N  B  Q  K  B  N  R  P  ...   r   n   b   q   k   b   n   
66597       22725  .  .  .  R  .  .  K  .  .  ...   .   .   r   .   .   .   k   
66598       13833  R  .  .  .  .  R  K  .  P  ...   r   .   .   q   .   r   k   

      p63 player  move  
0 